In [8]:


import numpy as np
import random
import math
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import time
import threading

# Distance function
def distance(pos1, pos2):
    x1, y1 = pos1
    x2, y2 = pos2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Duck class
class Duck:
    def __init__(self, unique_id, model, switchiness_mean, switchiness_sd):
        self.unique_id = unique_id
        self.model = model
        self.eating = 0
        self.pos = (random.randrange(model.width), random.randrange(model.height))
        self.switchiness = min(max(np.random.normal(switchiness_mean, switchiness_sd), 0), 1) # Trimmed between 0 and 1
        self.preference = 1 if distance(self.pos, (model.width * 0.3, model.height * 0.1)) < distance(self.pos, (model.width * 0.3, model.height * 0.8)) else 2

    def determine_preference(self):
        prev_preference = self.preference
        notzero = 0.00000001 # prevent division by zero
        if random.random() < self.switchiness:
            relative_prevalence1 = (self.model.total_food_source1 + notzero) / (self.model.total_food_source1 + self.model.total_food_source2 +  (notzero * 2))
            self.preference = 1 if random.random() < relative_prevalence1 else 2
        else:
            self.preference = prev_preference
    
    # Move function
    def move(self):
        possible_steps = [((self.pos[0] + dx) % self.model.width, 
                           (self.pos[1] + dy) % self.model.height) 
                          for dx in range(-1,2) for dy in range(-1,2)]
        possible_steps.remove(self.pos)
        
        # Get the duck-free steps.
        duck_free_steps = [step for step in possible_steps if self.model.is_duck_free(step)]
        
        if not duck_free_steps:
            return  # No place to move.
        
        if self.model.representational:
            self.determine_preference()
            preferred_source_y_range = self.model.feeder_1_y_range if self.preference == 1 else self.model.feeder_2_y_range
            food_positions = [food.pos for food in self.model.foods if food.pos[1] in preferred_source_y_range]
        else:
            food_positions = [food.pos for food in self.model.foods]
        
        if food_positions:
            nearest_food = min(food_positions, key=lambda pos: distance(self.pos, pos))
            
            # Filter the possible steps, choose only those that are duck-free.
            distances = [distance(step, nearest_food) for step in duck_free_steps]
            min_distance = min(distances)
            closest_steps = [step for step, dist in zip(duck_free_steps, distances) if dist == min_distance]
            
            if closest_steps:
                self.pos = random.choice(closest_steps)
        else:
            # If no food is available, move randomly.
            self.pos = random.choice(duck_free_steps)

    # Step function
    def step(self):
        if self.eating > 0:
            self.eating -= 1
            return
        food_at_pos = [food for food in self.model.foods if food.pos == self.pos]
        if food_at_pos:
            food_to_eat = food_at_pos[0]
            self.eating = food_to_eat.size
            self.model.foods.remove(food_to_eat)
        else:
            self.move()

# Food class
class Food:
    def __init__(self, unique_id, model, size):
        self.unique_id = unique_id
        self.model = model
        self.size = size
        self.pos = (random.choice(model.feeder_x_range), 
                    random.choice(random.choice([model.feeder_1_y_range, model.feeder_2_y_range])))

# Pond class (Model)
class Pond:
    def __init__(self, width, height, N, \
                 feeder1_rate, feeder1_size, feeder2_rate, feeder2_size, \
                switchiness_mean, switchiness_sd, representational=False):
        self.representational = representational
        self.width = width
        self.height = height
        self.num_ducks = N
        self.speed = initial_speed # default animation speed
        # self.ducks = [Duck(i, self) for i in range(N)]
        self.ducks = [Duck(i, self, switchiness_mean, switchiness_sd) for i in range(N)]
        self.foods = []
        self.feeder1_rate = feeder1_rate
        self.feeder1_size = feeder1_size
        self.feeder2_rate = feeder2_rate
        self.feeder2_size = feeder2_size
        self.feeder_x_range = range(int(0.3 * self.width), int(0.7 * self.width))
        self.feeder_1_y_range = range(int(0.1 * self.height), int(0.3 * self.height))
        self.feeder_2_y_range = range(int(0.7 * self.height), int(0.9 * self.height))

        self.total_food_source1 = 0
        self.total_food_source2 = 0

        self.timestep = 0
        self.uneaten1 = 0
        self.uneaten2 = 0
        self.crossed_midline = 0

    def is_duck_free(self, pos):
        return all(duck.pos != pos for duck in self.ducks)

    def step(self):
        self.uneaten1 = 0
        self.uneaten2 = 0
        for food in self.foods:
            if food.pos[1] in self.feeder_1_y_range:
                self.uneaten1 += food.size
            elif food.pos[1] in self.feeder_2_y_range:
                self.uneaten2 += food.size

        prev_duck_positions = [duck.pos for duck in self.ducks]
        random.shuffle(self.ducks)
        for duck in self.ducks:
            duck.step()
        
        # Check if ducks crossed the midline
        midline = self.height // 2
        self.crossed_midline = 0
        for prev_pos, duck in zip(prev_duck_positions, self.ducks):
            if (prev_pos[1] < midline and duck.pos[1] >= midline) or (prev_pos[1] >= midline and duck.pos[1] < midline):
                self.crossed_midline += 1

        if self.timestep % self.feeder1_rate == 0:
            self.drop_food(self.feeder_1_y_range, self.feeder1_size)
        if self.timestep % self.feeder2_rate == 0:
            self.drop_food(self.feeder_2_y_range, self.feeder2_size)
        self.timestep += 1
        
    def drop_food(self, y_range, size):
        food = Food(len(self.ducks) + len(self.foods), self, size)
        food.pos = (random.choice(self.feeder_x_range), random.choice(y_range))
        self.foods.append(food)
        if y_range == self.feeder_1_y_range:
            self.total_food_source1 += size
        else:
            self.total_food_source2 += size

    def plot_grid(self):
        grid = np.zeros((self.width, self.height), dtype=int)
        for food in self.foods:
            x, y = food.pos
            grid[x, y] = 3
        for duck in self.ducks:
            x, y = duck.pos
            grid[x, y] = 2 if duck.eating else 1
        return grid

# Parameters
pondwidth = 40
pondheight = 40
nducks = 50
rate1 = 4
rate2 = 8
size1 = 8
size2 = 8
timesteps = 100
switchiness_sd = 0.1
switchiness_mean = 0.5
initial_speed = 10
model = Pond(pondwidth, pondheight, nducks, rate1, rate2, size1, size2, \
            switchiness_mean, switchiness_sd, True)

# Plot function
def plot_grid(model):
    pond_grid = model.plot_grid()
    cmap = ListedColormap(['white', 'blue', 'violet', 'red'])
    plt.imshow(pond_grid.T, cmap=cmap, origin='lower', interpolation = 'none')
    plt.axhline(y=model.width / 2, color='grey', linestyle='--')

    # Counting how many ducks are closer to feeder1 and feeder2
    count_closer_to_feeder1 = sum(distance(duck.pos, (25, 5)) < distance(duck.pos, (25, 45)) for duck in model.ducks)
    count_closer_to_feeder2 = model.num_ducks - count_closer_to_feeder1
    proportion_closer_to_feeder1 = count_closer_to_feeder1 / model.num_ducks

    
    # Getting position and state of duck 1
    duck1_pos = model.ducks[0].pos
    duck1_state = "Eating" if model.ducks[0].eating > 0 else "Not Eating"
    duck1_eating = model.ducks[0].eating
    
    # Counting how many ducks are eating
    count_eating = sum(duck.eating > 0 for duck in model.ducks)
    
    # Calculating total uneaten food
    total_uneaten_food = sum(food.size for food in model.foods)
    
    # Calculating the sum of all duck states
    sum_duck_states = sum(duck.eating for duck in model.ducks)

    # concatenate duck states into a string
    duck_states_str = ''.join(str(duck.eating) for duck in model.ducks)

    # concatenate grid values -- giant string, but need for debugging color problem
    #tmpgrid = pond_grid.T
    grid_values_str = ''.join(str(val) + ('\n' if i % pondwidth == (pondwidth-1) else '') for i, val in enumerate(pond_grid.T.flatten()))
    grid_text_str = ""
    if show_ascii_plot_button.value:
        grid_text_str = grid_values_str.replace('0', ' ').replace('1', 'd').replace('2','X').replace('3','#')
    
    repnonrep = "NONrep"
    if model.representational:
        repnonrep = "REP"
        
    plt.title(f"Step: {model.timestep} Mode: {repnonrep}\nNear source 1: {count_closer_to_feeder1} = {proportion_closer_to_feeder1:.2f},\
                \nUneaten={total_uneaten_food} Eating={count_eating} State sum={sum_duck_states}\n\
                {duck_states_str}\n\
                Uneaten near source1: {model.uneaten1}, Uneaten near source2: {model.uneaten2}\n\
                Ducks crossed midline: {model.crossed_midline}\n\
                Duck 1: {duck1_pos},{duck1_eating},{duck1_state}\n\
                {grid_text_str}", fontsize=10, fontname='Courier')
   
    # Adding a light blue border around the grid
    for edge, spine in plt.gca().spines.items():
        spine.set_visible(True)
        spine.set_linewidth(2)
        spine.set_edgecolor('lightblue')

    plt.xticks([])  # Remove x-axis ticks
    plt.yticks([])  # Remove y-axis ticks
    plt.show()
        
paused = False

def toggle_pause(_):
    global paused
    paused = not paused
    pause_button.description = "Resume" if paused else "Pause"

pause_button = widgets.Button(description="Pause")
pause_button.on_click(toggle_pause)

current_speed = 0.1  # Declare a global variable for the current speed

# def update_speed(change):
#     global current_speed
#     current_speed = change.new  # Update the global variable
    
# Add sliders for parameters
rate1_slider = widgets.IntSlider(value=rate1, min=1, max=20, description='Rate 1:')
rate2_slider = widgets.IntSlider(value=rate2, min=1, max=20, description='Rate 2:')
size1_slider = widgets.IntSlider(value=size1, min=1, max=20, description='Size 1:')
size2_slider = widgets.IntSlider(value=size2, min=1, max=20, description='Size 2:')
speed_slider = widgets.IntSlider(value=initial_speed, min=1, max=100, description='Speed:')
#speed_slider.observe(update_speed, names='value')
timesteps_slider = widgets.IntSlider(value=timesteps, min=1, max=1000, description='Timesteps:')
show_ascii_plot_button = widgets.ToggleButton(value=False, description='Show text Plot', button_style='')



# Define the plot_output widget
plot_output = widgets.Output()

def run_simulation():
    global model
    model = Pond(pondwidth, pondheight, nducks, rate1, rate2, size1, size2, switchiness_mean, switchiness_sd, True)
    model.timestep = 0  # Resetting the timestep
    timesteps_from_slider = timesteps_slider.value  # Get the value from the slider
    for _ in range(timesteps_from_slider):
        while paused:  # Keep checking if the simulation is paused
            time.sleep(0.1)  # Sleep to prevent high CPU usage

        model.feeder1_rate = rate1_slider.value
        model.feeder2_rate = rate2_slider.value
        model.feeder1_size = size1_slider.value
        model.feeder2_size = size2_slider.value

        # Sleep to control animation speed
        time.sleep(speed_slider.value * 0.01)
        
        model.step()
        with plot_output:
            plot_grid(model)
            clear_output(wait=True)
        
def start_simulation_thread(change):
    run_simulation()

    
def start_simulation(change):
    simulation_thread = threading.Thread(target=run_simulation)
    simulation_thread.start()

start_button = widgets.Button(description="Start")
start_button.on_click(start_simulation_thread)
display(widgets.VBox([start_button, pause_button, rate1_slider, rate2_slider, size1_slider, size2_slider, speed_slider, timesteps_slider, show_ascii_plot_button, plot_output]))


# jim note to self: see widget-ducks-06.ipynb for previous version